### CUDA
在pytorch中，以下数据结构分为CPU和GPU两种：
1. Tensor
2. Variable（pytorch1.0后已经弃用）
3. Parameter
4. nn.Module（包括常用的layer，loss function， 以及Sequential）  

他们共性是都有一个`.cuda`方法，调用此方法即可将其转为对应的GPU对象。注意：
1. tensor.cuda和vairable.cuda会返回一个新对象
2. 这个新对象的数据已经转移至GPU，而之前的tensor/variable还在原来的设备上
3. module.cuda会将所有的数据都迁移至GPU，并返回自己。所以`module=module.cuda() `和`module.cuda()`是相同的效果


#### 首先检测CUDA是否可用

In [0]:
import torch as t
t.cuda.is_available()

True

In [0]:
import torch as t
a = t.Tensor(3, 4)
# 返回一个新的tensor，保存在第1块GPU上，但是原来的tensor并没有改变
a.cuda(0)
a.is_cuda

False

In [0]:
# 如果不指定cuda参数，那么默认存在第1块GPU上
a = a.cuda()
a.is_cuda

True

In [0]:
from torch import nn
module = nn.Linear(3,4)
# 对于模型来说，调用cuda方法会将自己的数据全部转移至GPU上
module.cuda()
module.weight.is_cuda

True

In [0]:
class VeryBigModule(nn.Module):
  def __init__(self):
    super()
    self.GiantParameter1 = nn.Parameter(t.randn(10000,20000)).cuda(0)
    self.GiantParameter2 = nn.Parameter(t.randn(20000,1000000)).cuda(1)
  def forward(self, x):
    x = self.GiantParameter1.mm(x.cuda(0))
    x = self.GiantParameter2.mm(x.cuda(1))
    return x

上面这部分，两个Parameter所占内存非常大，如果将这两个Parameter放到同一块GPU上运算，几乎会将Parameter占满，此时可以将其放在不同的GPU上运算。

**注意**：大部分的损失函数也都属于`nn.Module`，但在使用GPU时，如果我们忘记调用cuda方法，一般不会报错，但是**在某些情况下，会出现问题，所以应该记得要调用cuda方法**

In [0]:
# 交叉熵损失函数，带权重
criterion = nn.CrossEntropyLoss(weight = t.Tensor([1,3]))
input = t.randn(4,2, requires_grad=True).cuda()

# requires_grad_可以在原Tensor上直接修改，避免拷贝
target = t.Tensor([1,0,0,1]).long().cuda()

# loss = criterion(input, target)

criterion.cuda()
loss = criterion(input, target)
criterion._buffers

OrderedDict([('weight', tensor([1., 3.], device='cuda:0'))])

`loss = criterion(input, target)`这行会报错是因为`criterion`中有一个weight参数，并不在GPU上

除了调用对象`.cuda`方法，还可以**使用`torch.cuda.device`指定默认使用哪一块GPU**，或者使用**`torch.set_default_tensor_type`使程序默认使用GPU**，这样就不需要手动调用cuda切换GPU和CPU了

In [0]:
x = t.cuda.FloatTensor(2,3)

tensor([[0.0000e+00, 1.8750e+00, 1.4013e-45],
        [0.0000e+00, 0.0000e+00, 0.0000e+00]], device='cuda:0')

In [0]:
# 指定默认使用GPU 0
with t.cuda.device(0):
  a = t.cuda.FloatTensor(2,3)
  b = t.cuda.FloatTensor(2,3)
  
  c = a +b
  print(c)

tensor([[ 1.8750,  1.9844, -0.0196],
        [ 2.1439, -1.4802, -1.9669]], device='cuda:0')


In [0]:
# 指定默认tensor类型为GPU上的FloatTensor
t.set_default_tensor_type('torch.cuda.FloatTensor')
a = t.ones(2,3)
print(a.is_cuda, a.get_device())

True 0
